In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('recording17.csv', header=[0, 1], index_col=0)

# New Section

In [27]:
data

In [28]:
person_2 = data['2']
print(person_2)

In [29]:
# here I am extracting gaze estimates and speaking indicator for person '2'
gaze_x = person_2['gaze_x']
gaze_y = person_2['gaze_y']
speaking = person_2


In [30]:
total_rows = gaze_x.shape[0]
print(gaze_x)

In [ ]:
# this function is applying Gaussian kde
def plot_kde(x, y, label, title):
    # Removing invalid values
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]

    kde = gaussian_kde(np.vstack([x, y]))
    xi, yi = np.mgrid[x.min():x.max():x.size**0.5*1j, y.min():y.max():y.size**0.5*1j]
    zi = kde(np.vstack([xi.flatten(), yi.flatten()]))

    plt.figure(figsize=(8, 8))
    plt.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='auto', cmap='viridis')
    plt.title(title)
    plt.xlabel('Gaze X')
    plt.ylabel('Gaze Y')
    plt.legend()
    plt.show()

In [31]:
plot_kde(gaze_x, gaze_y, 'Gaze Estimates', 'Gaze KDE (g_0)')

In [ ]:
# funtion for plotting Conditional KDE for gaze estimates
def plot_conditional_kde(x, y, speaking, label, title):
    speaking_values = speaking['speaking'].unique()
    print(speaking_values)
    speaking_colors = ['r', 'g', 'b'][:len(speaking_values)]
    print(speaking_colors)
    plt.figure(figsize=(8, 8))

    for value, color in zip(speaking_values, speaking_colors):
        mask = speaking['speaking'] == value
        print(mask)
        x_subset, y_subset = x[mask], y[mask]
        print("x_sub",x_subset)
        # removing invalid values
        valid_values = np.isfinite(x_subset) & np.isfinite(y_subset)
        x_subset, y_subset = x_subset[valid_values], y_subset[valid_values]


        if len(x_subset) > 0 and len(y_subset) > 0:
            kde = gaussian_kde(np.vstack([x_subset, y_subset]))
            xi, yi = np.mgrid[x_subset.min():x_subset.max():x_subset.size**0.5*1j, y_subset.min():y_subset.max():y_subset.size**0.5*1j]
            zi = kde(np.vstack([xi.flatten(), yi.flatten()]))
            #print("\nzi:",zi)
            plt.title(title)
            plt.figure()
            plt.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='auto', cmap='viridis', alpha=0.5)
            #plt.scatter(x_subset, y_subset, c=color, s=5, label=f'Speaking: {value}')

    plt.title(title)
    plt.xlabel('Gaze X')
    plt.ylabel('Gaze Y')
    plt.legend()
    plt.show()

In [ ]:
# separating data where person 4 is speaking
gaze_person2_id4 = data['2'][data['4']['speaking'] == 1][['gaze_x', 'gaze_y', 'speaking']]

In [32]:
gaze_person2_id4['gaze_x']

In [33]:
plot_conditional_kde(gaze_person2_id4['gaze_x'], gaze_person2_id4['gaze_y'], gaze_person2_id4, 'Gaze Estimates', 'Conditional Gaze KDE (g_0|1)')

In [34]:
gaze_person2_id4

In [ ]:
# separating data where person 3 is speaking
gaze_person2_id3 = data['2'][data['3']['speaking'] == 1][['gaze_x', 'gaze_y', 'speaking']]

In [35]:
gaze_person2_id3

In [36]:
plot_conditional_kde(gaze_person2_id3['gaze_x'], gaze_person2_id3['gaze_y'], gaze_person2_id3, 'Gaze Estimates', ' conditional Gaze KDE (g_0|2)')

In [ ]:
# separating data where person 2 is speaking
gaze_person2_id1 = data['2'][data['1']['speaking'] == 1][['gaze_x', 'gaze_y', 'speaking']]

In [37]:
plot_conditional_kde(gaze_person2_id1['gaze_x'], gaze_person2_id1['gaze_y'], gaze_person2_id1, 'Gaze Estimates', 'Gaze KDE (g_0|3)')

In [ ]:
# Combining all the functions to calculate differnece map
# this function is taken from above section
def plot_kde(x, y, label, title):
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]

    kde = gaussian_kde(np.vstack([x, y]))
    xi, yi = np.mgrid[x.min():x.max():x.size**0.5*1j, y.min():y.max():y.size**0.5*1j]
    zi = kde(np.vstack([xi.flatten(), yi.flatten()]))

    plt.figure(figsize=(8, 8))
    plt.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='auto', cmap='viridis')
    plt.scatter(x, y, c='r', s=5, label=label)
    plt.title(title)
    plt.xlabel('Gaze X')
    plt.ylabel('Gaze Y')
    plt.legend()
    plt.show()

    return xi, yi, kde  # Return values to use in next function

# conditional kde function also taken from above section
def plot_conditional_kde(x, y, speaking, label, title):
    speaking_values = speaking['speaking'].unique()
    speaking_colors = ['r', 'g', 'b'][:len(speaking_values)]

    plt.figure(figsize=(8, 8))

    kde_list = []  # apending in kde list for later use

    for value, color in zip(speaking_values, speaking_colors):
        mask = speaking['speaking'] == value
        x_subset, y_subset = x[mask], y[mask]

        # Remove invalid values
        valid_values = np.isfinite(x_subset) & np.isfinite(y_subset)
        x_subset = x_subset[valid_values]
        y_subset = y_subset[valid_values]


        if len(x_subset) > 0 and len(y_subset) > 0:
            kde = gaussian_kde(np.vstack([x_subset, y_subset]))
            xi_conditional, yi_conditional = np.mgrid[x_subset.min():x_subset.max():x_subset.size**0.5*1j,
                                                      y_subset.min():y_subset.max():y_subset.size**0.5*1j]
            zi_conditional = kde(np.vstack([xi_conditional.flatten(), yi_conditional.flatten()]))

            plt.pcolormesh(xi_conditional, yi_conditional, zi_conditional.reshape(xi_conditional.shape),
                           shading='auto', cmap='viridis', alpha=0.5)
            plt.scatter(x_subset, y_subset, c=color, s=5, label=f'Speaking: {value}')

            kde_list.append(kde)  # Appending kde to the list

    plt.title(title)
    plt.xlabel('Gaze X')
    plt.ylabel('Gaze Y')
    plt.legend()
    plt.show()

    return xi_conditional, yi_conditional, kde_list  # Return the list of kde

# Subtracting
def subtract_kde(general_kde, conditional_kde_list, xi, yi, x, y, label, title):
    # Evaluate both KDEs at the given set of points
    general_density = general_kde(np.vstack([xi.flatten(), yi.flatten()]))
    x = np.asarray(x).flatten()
    y = np.asarray(y).flatten()

    plt.figure(figsize=(8, 8))

    for conditional_kde in conditional_kde_list:
        conditional_density = conditional_kde(np.vstack([xi.flatten(), yi.flatten()]))
        difference_density = conditional_density - general_density
        difference_density = difference_density.reshape(xi.shape)

        plt.pcolormesh(xi, yi, difference_density, shading='auto', cmap='viridis')

        plt.title(title)
        plt.xlabel('Gaze X')
        plt.ylabel('Gaze Y')



In [38]:
# Calling plot function
xi_general, yi_general, general_kde = plot_kde(gaze_x, gaze_y, 'General KDE', 'General KDE Plot')

# calling conditional function
xi_conditional, yi_conditional, conditional_kde_list = plot_conditional_kde(gaze_person2_id1['gaze_x'], gaze_person2_id1['gaze_y'], gaze_person2_id1, 'Conditional KDE', 'Conditional KDE Plot')

# calling subtract function
subtract_kde(general_kde, conditional_kde_list, xi_general, yi_general, gaze_person2_id1['gaze_x'], gaze_person2_id1['gaze_y'], 'Subtracted KDE', 'Subtracted KDE Plot')




In [39]:
# Calling plot function
xi_general, yi_general, general_kde = plot_kde(gaze_x, gaze_y, 'General KDE', 'General KDE Plot')

# calling conditional function
xi_conditional, yi_conditional, conditional_kde_list = plot_conditional_kde(gaze_person2_id3['gaze_x'], gaze_person2_id3['gaze_y'], gaze_person2_id3, 'Conditional KDE', 'Conditional KDE Plot')

# calling subtract function
subtract_kde(general_kde, conditional_kde_list, xi_general, yi_general, gaze_person2_id3['gaze_x'], gaze_person2_id3['gaze_y'], 'Subtracted KDE', 'Subtracted KDE Plot')




In [40]:
# Calling plot function
xi_general, yi_general, general_kde = plot_kde(gaze_x, gaze_y, 'General KDE', 'General KDE Plot')

# calling conditional function
xi_conditional, yi_conditional, conditional_kde_list = plot_conditional_kde(gaze_person2_id4['gaze_x'], gaze_person2_id4['gaze_y'], gaze_person2_id4, 'Conditional KDE', 'Conditional KDE Plot')

# calling subtract function
subtract_kde(general_kde, conditional_kde_list, xi_general, yi_general, gaze_person2_id4['gaze_x'], gaze_person2_id4['gaze_y'], 'Subtracted KDE', 'Subtracted KDE Plot')


